# 0. Imports

In [1]:
from sklearn.linear_model import LinearRegression
from covid_project.data_utils import clean_covid_data, clean_policy_data, prep_policy_data
from covid_project.policy_mappings import policy_dict_v1
import pandas as pd
import statsmodels.api as sm
import os

# 1. Fit a simple model

In [95]:
# policy_name = 'manufacturing - start - state'
# bins_list = [(0, 7), (8, 14), (15, 28), (29, 60), (61, 999)]

# filename = policy_name.replace(" - ", "_") +\
#                 "-bins=" + ''.join([str(b[0])+"-"+str(b[1])+"_" for b in bins_list])[:-1] + ".csv"

# root_path = "./data/single_policy_bins/"

# data = pd.read_csv(root_path + filename, header=[0, 1], index_col=0)

def get_processed_data(policy_name,
                       bins_list,
                       root_path="./data/single_policy_bins/",):
    filename = policy_name.replace(" - ", "_") +\
                "-bins=" + ''.join([str(b[0])+"-"+str(b[1])+"_" for b in bins_list])[:-1] + ".csv"
    path = root_path + filename
    
    if os.path.exists(path):
        data = pd.read_csv(root_path + filename, header=[0, 1], index_col=0)
        return True, data
    else:
        return False, None

suc, data = get_processed_data('manufacturing - start - state',
                          bins_list = [(0, 7), (8, 14), (15, 28), (29, 60), (61, 999)])

In [97]:
data

info                                              \
        location_type    state   county        date new_cases_1e6   
69440          county  Alabama  autauga  2020-01-22          0.00   
69441          county  Alabama  autauga  2020-01-23          0.00   
69442          county  Alabama  autauga  2020-01-24          0.00   
69443          county  Alabama  autauga  2020-01-25          0.00   
69444          county  Alabama  autauga  2020-01-26          0.00   
...               ...      ...      ...         ...           ...   
2798268        county  Wyoming   weston  2021-12-26          0.00   
2798269        county  Wyoming   weston  2021-12-27         14.08   
2798270        county  Wyoming   weston  2021-12-28         28.17   
2798271        county  Wyoming   weston  2021-12-29         42.25   
2798272        county  Wyoming   weston  2021-12-30         28.17   

                                                               \
        new_deaths_1e6 new_cases_7day_1e6 new_deaths_7day_1e6   
69440              0.0           0.000000                 0.0   
69441              0.0           0.000000                 0.0   
69442              0.0           0.000000                 0.0   
69443              0.0           0.000000                 0.0   
69444              0.0           0.000000                 0.0   
...                ...                ...                 ...   
2798268            0.0          20.140845                 0.0   
2798269            0.0          12.112676                 0.0   
2798270            0.0          12.112676                 0.0   
2798271            0.0          18.169014                 0.0   
2798272            0.0          16.056338                 0.0   

        manufacturing - start - state                          
                                  0-7 8-14 15-28 29-60 61-999  
69440                               0    0     0     0      0  
69441                               0    0     0     0      0  
69442                               0    0     0     0      0  
69443                               0    0     0     0      0  
69444                               0    0     0     0      0  
...                               ...  ...   ...   ...    ...  
2798268                             0    0     0     0      0  
2798269                             0    0     0     0      0  
2798270                             0    0     0     0      0  
2798271                             0    0     0     0      0  
2798272                             0    0     0     0      0  

[2228387 rows x 13 columns]

In [58]:
data[policy_name]

,0-7,8-14,15-28,29-60,61-999
69440,0,0,0,0,0
69441,0,0,0,0,0
69442,0,0,0,0,0
69443,0,0,0,0,0
69444,0,0,0,0,0
...,...,...,...,...,...
2798268,0,0,0,0,0
2798269,0,0,0,0,0
2798270,0,0,0,0,0
2798271,0,0,0,0,0


In [62]:
dep_var = 'new_cases_1e6'
y = data[('info', 'new_cases_1e6')]
X = data[policy_name]
X = sm.add_constant(X)

In [63]:
X

,const,0-7,8-14,15-28,29-60,61-999
69440,1.0,0,0,0,0,0
69441,1.0,0,0,0,0,0
69442,1.0,0,0,0,0,0
69443,1.0,0,0,0,0,0
69444,1.0,0,0,0,0,0
...,...,...,...,...,...,...
2798268,1.0,0,0,0,0,0
2798269,1.0,0,0,0,0,0
2798270,1.0,0,0,0,0,0
2798271,1.0,0,0,0,0,0


In [15]:
y

69440       0.00
69441       0.00
69442       0.00
69443       0.00
69444       0.00
           ...  
2798268     0.00
2798269    14.08
2798270    28.17
2798271    42.25
2798272    28.17
Name: (info, new_cases_1e6), Length: 2228387, dtype: float64

In [19]:
model = sm.OLS(y, X)
results = model.fit()

In [36]:
results.params

manufacturing - start - state_0-7       10.754544
manufacturing - start - state_8-14      13.782788
manufacturing - start - state_15-28     14.436798
manufacturing - start - state_29-60     16.115160
manufacturing - start - state_61-999    30.922036
dtype: float64

In [42]:
results.__dict__.keys()

dict_keys(['_results', '__doc__'])

In [53]:
results._results._cache.keys()

dict_keys(['wresid', 'eigenvals', 'condition_number', 'ssr', 'uncentered_tss', 'rsquared', 'nobs', 'rsquared_adj', 'ess', 'mse_model', 'mse_resid', 'fvalue', 'f_pvalue', 'llf', 'aic', 'bic', 'scale', 'bse', 'tvalues', 'pvalues'])

In [81]:
results._results.__dict__.keys()

dict_keys(['params', 'model', 'k_constant', '_data_attr', '_data_in_cache', 'normalized_cov_params', '_use_t', '_cache', '_wexog_singular_values', 'df_model', 'df_resid', 'cov_type', 'cov_kwds', 'diagn'])

In [85]:
dir(results)

['HC0_se',
 'HC1_se',
 'HC2_se',
 'HC3_se',
 '_HCCM',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abat_diagonal',
 '_cache',
 '_data_attr',
 '_data_in_cache',
 '_get_robustcov_results',
 '_is_nested',
 '_use_t',
 '_wexog_singular_values',
 'aic',
 'bic',
 'bse',
 'centered_tss',
 'compare_f_test',
 'compare_lm_test',
 'compare_lr_test',
 'condition_number',
 'conf_int',
 'conf_int_el',
 'cov_HC0',
 'cov_HC1',
 'cov_HC2',
 'cov_HC3',
 'cov_kwds',
 'cov_params',
 'cov_type',
 'df_model',
 'df_resid',
 'diagn',
 'eigenvals',
 'el_test',
 'ess',
 'f_pvalue',
 'f_test',
 'fittedvalues',
 'fvalue',
 'get_influence',
 'get_prediction',
 'get_robustcov_results',
 'info_c

In [83]:
results._results._data_in_cache

['fittedvalues', 'resid', 'wresid']

In [60]:
results.pvalues.to_dict()

{'manufacturing - start - state_0-7': 7.018221353441827e-89,
 'manufacturing - start - state_8-14': 7.069141936740188e-127,
 'manufacturing - start - state_15-28': 6.6695191796244436e-276,
 'manufacturing - start - state_29-60': 0.0,
 'manufacturing - start - state_61-999': 0.0}

In [47]:
results.df_model

5.0

In [37]:
results.bse

manufacturing - start - state_0-7       0.538028
manufacturing - start - state_8-14      0.575177
manufacturing - start - state_15-28     0.406711
manufacturing - start - state_29-60     0.269014
manufacturing - start - state_61-999    0.066949
dtype: float64

In [40]:
results.rsquared

0.08953946831763915

In [23]:
summary = results.summary()
summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                    OLS Regression Results                                    
==============================================================================================
Dep. Variable:     ('info', 'new_cases_1e6')   R-squared (uncentered):                   0.090
Model:                                   OLS   Adj. R-squared (uncentered):              0.090
Method:                        Least Squares   F-statistic:                          4.383e+04
Date:                       Sat, 11 Jun 2022   Prob (F-statistic):                        0.00
Time:                               12:26:20   Log-Likelihood:                     -1.2309e+07
No. Observations:                    2228387   AIC:                                  2.462e+07
Df Residuals:                        2228382   BIC:                                  2.462e+07
Df Model:                                  5                                                  
Covariance Type:                   nonrobust                                                  
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
manufacturing - start - state_0-7       10.7545      0.538     19.989      0.000       9.700      11.809
manufacturing - start - state_8-14      13.7828      0.575     23.963      0.000      12.655      14.910
manufacturing - start - state_15-28     14.4368      0.407     35.496      0.000      13.640      15.234
manufacturing - start - state_29-60     16.1152      0.269     59.904      0.000      15.588      16.642
manufacturing - start - state_61-999    30.9220      0.067    461.875      0.000      30.791      31.053
==============================================================================
Omnibus:                  5283027.126   Durbin-Watson:                   1.402
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     417790315089.813
Skew:                          23.189   Prob(JB):                         0.00
Kurtosis:                    2123.731   Cond. No.                         8.59
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [90]:
def fit_ols_model_single_policy(data,
                                policy_name,
                                dep_var,
                                use_const=True):

    """Fit an ols model from statsmodels
    
    Parameters
    ----------
    data
    
    policy_name
    
    dep_var
    
    use_const
    
    Returns
    ---------
    dictionary containing the coefficience (params), standard error of the coefficients (std_err),
    r^2 value (r_squared) and the p values (p_values)
    """
    y = data[('info', dep_var)]
    X = data[policy_name]
    
    if use_const:
        X = sm.add_constant(X)

    model = sm.OLS(y, X)
    results = model.fit()

    results_dict = {
        'r_squared': results.rsquared,
        'p_values': results.pvalues.to_dict(),
        'params': results.params.to_dict(),
        'std_err': results.bse.to_dict()
    }
    
    return results_dict

In [91]:
res = fit_ols_model_single_policy(data,
                policy_name,
                'new_cases_1e6',
                True)

In [88]:
res

{'r_squared': 0.007396003861014577,
 'p_values': {'const': 0.0,
  '0-7': 1.5385998858129313e-91,
  '8-14': 8.139974769145486e-42,
  '15-28': 5.459562113357568e-68,
  '29-60': 1.8709101287614066e-86,
  '61-999': 0.0},
 'params': {'const': 21.31820959496815,
  '0-7': -10.56366564411765,
  '8-14': -7.53542176528055,
  '15-28': -6.881411953416204,
  '29-60': -5.20304911135131,
  '61-999': 9.603826833860145},
 'std_err': {'const': 0.05097110775154247,
  '0-7': 0.5205797232678747,
  '8-14': 0.5561895787275343,
  '15-28': 0.39493347624033953,
  '29-60': 0.2640063492389575,
  '61-999': 0.08218260084002933}}

# 2. Fit multiple models

get all policy names

In [31]:
def get_all_policies(policy_dict, min_samples):
    policy_data = clean_policy_data()
    policy_data_prepped = prep_policy_data(policy_data=policy_data,
                                           policy_dict=policy_dict,
                                           min_samples=min_samples)
    all_policies = policy_data_prepped['full_policy'].unique()
    return all_policies

all_policies = get_all_policies(policy_dict_v1, 3)

In [101]:
results = dict()
from tqdm.notebook import tqdm
for policy in tqdm(all_policies):
    suc, data = get_processed_data(policy, bins_list)
    res = fit_ols_model_single_policy(data,
                                      policy,
                                      'new_cases_1e6',
                                      True,)
    results[policy] = res

  0%|          | 0/50 [00:00<?, ?it/s]

In [102]:
results

{'outdoor and recreation - stop - county': {'r_squared': 4.051495902146485e-05,
  'p_values': {'const': 0.0,
   '0-7': 8.293513825220407e-06,
   '8-14': 0.0008766474070521422,
   '15-28': 0.08208336956932982,
   '29-60': 0.09726951966165569,
   '61-999': 2.414554897300173e-13},
  'params': {'const': 24.544402369635748,
   '0-7': 18.84903513036451,
   '8-14': 15.041728582745264,
   '15-28': 5.558097630364793,
   '29-60': -3.5064987238024554,
   '61-999': 4.0912783178358145},
  'std_err': {'const': 0.03936045994332847,
   '0-7': 4.228636048168299,
   '8-14': 4.520577608118154,
   '15-28': 3.196652245461495,
   '29-60': 2.1145927838053167,
   '61-999': 0.5586417527211902}},
 'non-essential businesses - stop - state': {'r_squared': 0.030231830752811883,
  'p_values': {'const': 0.0,
   '0-7': 9.405812061575971e-18,
   '8-14': 4.328922614629604e-08,
   '15-28': 1.9910929471460394e-11,
   '29-60': 6.433177178181158e-174,
   '61-999': 0.0},
  'params': {'const': 9.928571048368108,
   '0-7': 3.